In [55]:
from tika import parser
from __future__ import print_function
import fitz
import sys, time

In [128]:
# Support Code

# For getting out a particular image.
def recoverpix(doc, item):
    x = item[0]  # xref of PDF image
    s = item[1]  # xref of its /SMask
    pix1 = fitz.Pixmap(doc, x)
    if s == 0:                    # has no /SMask
        return pix1               # no special handling
    pix2 = fitz.Pixmap(doc, s)    # create pixmap of /SMask entry
    # check that we are safe
    if not (pix1.irect == pix2.irect and \
            pix1.alpha == pix2.alpha == 0 and \
            pix2.n == 1):
        print("pix1", pix1, "pix2", pix2)
        raise ValueError("unexpected situation")
    pix = fitz.Pixmap(pix1)       # copy of pix1, alpha channel added
    pix.setAlpha(pix2.samples)    # treat pix2.samples as alpha value
    pix1 = pix2 = None            # free temp pixmaps
    return pix

# To process the text you get from the parser
def processRawText(raw):
    raw_text_data = raw["content"]
    lines = raw_text_data.split("\n")
    pageContent = []
    for line in lines:
        if len(line)>3:
            if line == 'f r o m  o l d  b o o k s  o r  ':
                line = "from old book or"
            sp = line.split()
            line = " ".join(sp)
            pageContent.append(line)
    return pageContent

# Take out the images from a pdf file and save them at the "saveLocation"
# Return : the count of the images
## Reference : https://github.com/rk700/PyMuPDF/blob/master/demo/extract-img1.py
def imageExtracter(pdfFile,saveLocation):
    
    t0 = time.clock()
    doc = fitz.open(pdfFile)           # the PDF
    imgcount = 0                           # counts extracted images
    xreflist = []                          # records images already extracted
    lenXREF = doc._getXrefLength()         # only used for information

    # display some file info
    print("file: %s, pages: %s, objects: %s" % (fileName, len(doc), lenXREF-1))

    for i in range(len(doc)):
        imglist = doc.getPageImageList(i)
        for img in imglist:
            if img[0] in xreflist:         # this image has been processed
                continue 
            xreflist.append(img[0])        # take note of the xref
            pix = recoverpix(doc, img[:2]) # make pixmap from image
            if pix.n - pix.alpha < 4:      # can be saved as PNG
                pass
            else:                          # must convert CMYK first
                pix0 = fitz.Pixmap(fitz.csRGB, pix)
                pix = pix0
            pix.writePNG(saveLocation+"p%i-%s.png" % (i, img[7]))
            imgcount += 1
            pix = None                     # free Pixmap resources

    t1 = time.clock()
    print("run time", round(t1-t0, 2))
    print("extracted images", imgcount)
    return imgcount

#Write Text file to a location

def fileWriter(listText,location):
    f = open(location,"w")
    Lines = ""
    for line in listText:
        Lines += line + " "
    f.write(Lines)
    f.close()
    

In [131]:
# foldLoc = "/home/sp/nlpfinal/NCERT_Data/textcontent/"
# listOfdir = os.listdir(foldLoc)

# targetLoc = "/home/sp/nlpfinal/NCERT_Data/extracted_data/"
# targetLoc = "/home/sp/nlpfinal/NCERT_Data/extracted_data/"

# for working_dir in listOfdir:
#     sourceLocation = foldLoc + working_dir + "/"
#     targetDir = targetLoc + "textData/" + working_dir + "/"
#     targetDirImg = targetLoc + "images/" + working_dir + "/"
#     if not os.path.exists(targetDir):
#         os.makedirs(targetDir)
#     listOfFiles = os.listdir(sourceLocation)
#     for file in listOfFiles:
#         fileLoc = sourceLocation + file
#         print(fileLoc)
#         raw = parser.from_file(fileLoc)
#         textList = processRawText(raw)
#         targetFileLocation = targetDir + (file.split("."))[0] + ".txt"
#         print(targetFileLocation)
#         fileWriter(textList,targetFileLocation)
#         imageLocation = targetDirImg + (file.split("."))[0] + "/"
#         print(imageLocation)
#         if not os.path.exists(imageLocation):
#             os.makedirs(imageLocation)
#         imageExtracter(fileLoc,imageLocation)
        
        
    

In [130]:
targetDir

'/home/sp/nlpfinal/NCERT_Data/extracted_data/textData/class1/'

In [114]:
a = "Raja.pdf"
(a.split("."))[0]

'Raja'

In [115]:
listOfdir[0:2]

['class10', 'class8']

In [117]:
ff = open("/home/sp/nlpfinal/NCERT_Data/extracted_data/textData/class10/jeff110.txt","r")

In [118]:
text = ff.read()

In [119]:
import nltk

In [120]:
ff = nltk.sent_tokenize(text)

In [122]:
ff

['Chapter 10 BEFORE YOU READ Activity Use a dictionary or ask for your teacher’s help as you discuss the following questions in groups.',
 '1.',
 'What is a sermon?',
 'Is it different from a lecture or a talk?',
 'Can this word also be used in a negative way or as a joke (as in “my mother’s sermon about getting my work done on time…”)?',
 '2.',
 'Find out the meanings of the words and phrases given in the box.',
 'afflicted with be composed desolation, lamentation procure be subject to 3.',
 'Have you heard of the Sermon on the Mount?',
 'Who delivered it?',
 'Who do you think delivered a sermon at Benares?',
 'GAUTAMA Buddha (563 B.C.',
 '– 483 B.C.)',
 'began life as a prince named Siddhartha Gautama, in northern India.',
 'At twelve, he was sent away for schooling in the Hindu sacred scriptures and four years later he returned home to marry a princess.',
 'They had a son and lived for ten years as befitted royalty.',
 'At about the age of twenty-five, the Prince, heretofore shielde